In [ ]:
!pip install -U albumentations==0.4.5
!pip install torch==1.7.1 torchvision==0.8.2
!pip install segmentation-models-pytorch
!wget https://download.openmmlab.com/mmcv/dist/1.3.0/torch1.7.0/cu102/mmcv_full-1.3.0%2Btorch1.7.0%2Bcu102-cp37-cp37m-manylinux1_x86_64.whl
!pip install ./mmcv_full-1.3.0+torch1.7.0+cu102-cp37-cp37m-manylinux1_x86_64.whl
!pip install ../input/mmdetectionv260/addict-2.4.0-py3-none-any.whl
!pip install ../input/mmdetectionv260/mmpycocotools-12.0.3-cp37-cp37m-linux_x86_64.whl
!git clone https://github.com/shinya7y/UniverseNet
!cp -r UniverseNet/* ./
!pip install -v -e .
!cp ../input/molconfig/uniconfig.py configs/universenet/universenet101.py

In [ ]:
import mmcv
import os
from tqdm.auto import tqdm
import segmentation_models_pytorch as smp
print(mmcv.__version__)
import copy
import os.path as osp
import glob
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
import albumentations as albu
# Check MMDetection installation
import mmdet
print(mmdet.__version__)

# Check mmcv installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
from mmcv import Config
print(get_compiling_cuda_version())
print(get_compiler_version())

from mmdet.datasets import build_dataset, CocoDataset
from mmdet.models import build_detector
from mmdet.datasets.builder import DATASETS
from mmdet.datasets.custom import CustomDataset
from mmdet.apis import train_detector, set_random_seed, init_detector#, inference_detector

from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import warnings

import mmcv
import numpy as np
import torch
from mmcv.ops import RoIPool
from mmcv.parallel import collate, scatter
from mmcv.runner import load_checkpoint

from mmdet.core import get_classes
from mmdet.datasets import replace_ImageToTensor
from mmdet.datasets.pipelines import Compose
from mmdet.models import build_detector
from IPython.core.display import display, HTML
import pickle
import cv2
from matplotlib import pyplot as plt
import numpy as n
from skimage.segmentation import mark_boundaries
from skimage.measure import label, regionprops
from collections import Counter
import numpy as np
from matplotlib import pyplot as plt
from scipy.spatial import distance
from torch.utils.data import TensorDataset, DataLoader, Dataset


In [ ]:
def inference_detector(model, imgs,rot90=True):
    """Inference image(s) with the detector.
    Args:
        model (nn.Module): The loaded detector.
        imgs (str/ndarray or list[str/ndarray] or tuple[str/ndarray]):
           Either image files or loaded images.
    Returns:
        If imgs is a list or tuple, the same length list type results
        will be returned, otherwise return the detection results directly.
    """

    if isinstance(imgs, (list, tuple)):
        is_batch = True
    else:
        imgs = [imgs]
        is_batch = False

    cfg = model.cfg
    device = next(model.parameters()).device  # model device

    if isinstance(imgs[0], np.ndarray):
        cfg = cfg.copy()
        # set loading pipeline type
        cfg.data.test.pipeline[0].type = 'LoadImageFromWebcam'

    cfg.data.test.pipeline = replace_ImageToTensor(cfg.data.test.pipeline)
    test_pipeline = Compose(cfg.data.test.pipeline)

    datas = []
    for img in imgs:
        # prepare data
        if isinstance(img, np.ndarray):
            # directly add img
            data = dict(img=img)
        else:
            # add information into dict
            data = dict(img_info=dict(filename=img), img_prefix=None)
        # build the data pipeline
        data = test_pipeline(data)
        datas.append(data)

    data = collate(datas, samples_per_gpu=len(imgs))
    # just get the actual data from DataContainer
    data['img_metas'] = [img_metas.data[0] for img_metas in data['img_metas']]
#     if data['img'][0].data[0].shape[-1] > data['img'][0].data[0].shape[-2]:
#         data['img'] = [img.data[0] for img in data['img']]
#     else:
#         if rot90:
#             data['img'] = [torch.rot90(img.data[0],1,[-2, -1]) for img in data['img']]
#         else:
    data['img'] = [img.data[0] for img in data['img']]
        
    if next(model.parameters()).is_cuda:
        # scatter to specified GPU
        data = scatter(data, [device])[0]
    else:
        for m in model.modules():
            assert not isinstance(
                m, RoIPool
            ), 'CPU inference with RoIPool is not supported currently.'

    # forward the model
    with torch.no_grad():
        results = model(return_loss=False, rescale=True, **data)

    if not is_batch:
        return results[0]
    else:
        return results
def norm(img):
    img-=img.min()
    return img/img.max()
def nms(dets, scores, thresh):
    '''
    dets is a numpy array : num_dets, 4
    scores ia  nump array : num_dets,
    '''
    x1 = dets[:, 0]
    y1 = dets[:, 1]
    x2 = dets[:, 2]
    y2 = dets[:, 3]

    areas = (x2 - x1 + 1) * (y2 - y1 + 1)
    order = scores.argsort()[::-1] # get boxes with more ious first

    keep = []
    while order.size > 0:
        i = order[0] # pick maxmum iou box
        keep.append(i)
        xx1 = np.maximum(x1[i], x1[order[1:]])
        yy1 = np.maximum(y1[i], y1[order[1:]])
        xx2 = np.minimum(x2[i], x2[order[1:]])
        yy2 = np.minimum(y2[i], y2[order[1:]])

        w = np.maximum(0.0, xx2 - xx1 + 1) # maximum width
        h = np.maximum(0.0, yy2 - yy1 + 1) # maxiumum height
        inter = w * h
        ovr = inter / (areas[i] + areas[order[1:]] - inter)

        inds = np.where(ovr <= thresh)[0]
        order = order[inds + 1]

    return keep
def bb_intersection_over_union(boxA, boxB):
    # determine the (x, y)-coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    # compute the area of intersection rectangle
    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
    # compute the area of both the prediction and ground-truth
    # rectangles
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = interArea / float(boxAArea + boxBArea - interArea)
    # return the intersection over union value
    return iou
def load_obj(name ):
    with open(name, 'rb') as f:
        return pickle.load(f)
def to_tensor(x, **kwargs):
    return x.transpose(2, 0, 1).astype('float32')

def get_valid_augmentation(size=512):
    return albu.Compose([
        albu.Resize(size,size,always_apply=True),
    ])


def get_preprocessing(preprocessing_fn):
    _transform = [
        albu.Lambda(image=preprocessing_fn),
        albu.Lambda(image=to_tensor, mask=to_tensor),
    ]
    return albu.Compose(_transform)

class MolDataset(Dataset):
    def __init__(self, paths, transforms=None, preprocessing=None,mode='test'):
        self.paths = paths
        self.transforms = transforms
        self.preprocessing = preprocessing
        self.mode = mode
    def __getitem__(self, idx):
        p = self.paths[idx]
        img = cv2.imread(p,0)
        noise_img = cv2.cvtColor(img.astype(np.uint8),cv2.COLOR_GRAY2RGB)
        if self.transforms:
            augmented = self.transforms(image=noise_img)
            noise_img = augmented['image']
        if self.preprocessing:
            preprocessed = self.preprocessing(image=noise_img)
            noise_img = preprocessed['image']
        return noise_img,p

    def __len__(self):
        return len(self.paths)
def predict_one_image(config,img_real):
    config['model'].eval()
#     img_real = cv2.imread(path,0)
    img = cv2.cvtColor(img_real.astype(np.uint8),cv2.COLOR_GRAY2RGB)
    if config['aug']:
        augmented = config['aug'](image=img)
        img = augmented['image']
    if config['preprocessing']:
        preprocessed = config['preprocessing'](image=img)
        img = preprocessed['image']
    x = torch.from_numpy(np.expand_dims(img,0)).to(config['d'])
    with torch.no_grad():
        predicted,p_class = config['model'](x)
        predicted_sigmoid = torch.sigmoid(predicted)
        fet = config['model'].encoder.extract_features(x)
        avg_features = torch.nn.AvgPool2d([fet.shape[-2],fet.shape[-1]])(fet)
    mask = predicted_sigmoid[0].cpu().numpy().transpose(1,2,0)
    mask = cv2.resize(mask,(img_real.shape[1],img_real.shape[0]))
    return mask

def predict_one_image_with_crop(config,img_real):
    config['model'].eval()
#     img_real = cv2.imread(path,0)
    h,w = img_real.shape[0:2]
    flag = h>w
    m = max(h,w)
    c = min(h,w)
    masks = np.zeros([h,w,3])
    x_flag = False
    y_flag = False
    for row in range(0,m,c):
        if flag:
            img = cv2.cvtColor(img_real[row:row+c].astype(np.uint8),cv2.COLOR_GRAY2RGB)
            if img.shape[0] != img.shape[1]:
                img = cv2.cvtColor(img_real[-c:].astype(np.uint8),cv2.COLOR_GRAY2RGB)
                y_flag = True
        else:
            img = cv2.cvtColor(img_real[:,row:row+c].astype(np.uint8),cv2.COLOR_GRAY2RGB)
            if img.shape[0] != img.shape[1]:
                img = cv2.cvtColor(img_real[:,-c:].astype(np.uint8),cv2.COLOR_GRAY2RGB)
                x_flag = True
        if config['aug']:
            augmented = config['aug'](image=img)
            img = augmented['image']
        if config['preprocessing']:
            preprocessed = config['preprocessing'](image=img)
            img = preprocessed['image']
        x = torch.from_numpy(np.expand_dims(img,0)).to(config['d'])
        with torch.no_grad():
            predicted,p_class = config['model'](x)
            predicted_sigmoid = torch.sigmoid(predicted)
            fet = config['model'].encoder.extract_features(x)
            avg_features = torch.nn.AvgPool2d([fet.shape[-2],fet.shape[-1]])(fet)
        mask = predicted_sigmoid[0].cpu().numpy().transpose(1,2,0)
        mask = cv2.resize(mask,(c,c))
        if flag:
            if y_flag:
                masks[-c:] = mask
            else:
                masks[row:row+c] = mask
        else:
            if x_flag:
                masks[:,-c:] = mask
            else:
                masks[:,row:row+c] = mask
    return masks

In [ ]:
classes = load_obj('../input/csv2kaggle/data/mapper.pkl') 
@DATASETS.register_module()
class MolLatter2(CocoDataset):
    CLASSES = tuple([str(row) for row in classes.keys()])
classes

In [ ]:
config_file = 'configs/universenet/universenet101.py'
checkpoint_file = '../input/fork-of-step2-universenet20-08-101/epoch_1.pth'
device = 'cuda:0'
# init a detector
model = init_detector(config_file, checkpoint_file, device=device)
# inference the demo image


In [ ]:
paths = glob.glob('../input/bms-molecular-translation/test/0/0/0/*.png')
df = pd.read_csv("../input/bms-molecular-translation/train_labels.csv")
config_b5 = {
    "backbone":"efficientnet-b5",
    "im_size":384,
    "model_path":"../input/moleffunet/best_unet-b5.pth",
    "seg_calss": 3,
    "cls_class":12,
    "d":'cuda:0'}
config_b3 = {
    "backbone":"efficientnet-b3",
    "im_size":512,
    "model_path":"../input/moleffunet/best_unet-b3.pth",
    "seg_calss": 3,
    "cls_class":12,
    "d":'cuda:0'}
class config:
    real_path="../input/bms-molecular-translation/train"
    mask_path='../input/mol-temp'
    font = cv2.FONT_HERSHEY_SIMPLEX
    classes = ('1', '35', '5', '7', '8', '9', '14', '15', '16', '17', '53')
    colors = [(255,0,0),(0,255,0),(0,0,255),(255,0,255),(255,255,0),(0,255,255),(255,255,255),(255,127,255),(255,255,127),(127,255,255),(127,127,127)]
    unet = [config_b5,config_b3]
    mask_TH = 0.5
    bbox_TH = 0.2

In [ ]:
for i in range(len(config.unet)):
    aux_params=dict(pooling='avg',dropout=0.2,activation="sigmoid",classes=config.unet[i]['cls_class'])
    config.unet[i]['model'] = torch.nn.DataParallel(smp.Unet(config.unet[i]['backbone'], classes=config.unet[i]['seg_calss'],encoder_weights=None, aux_params=aux_params))
    config.unet[i]['model'].load_state_dict(torch.load(config.unet[i]['model_path'],map_location='cpu'))
    config.unet[i]['model'] = config.unet[i]['model'].module
    config.unet[i]['model'].to(config.unet[i]['d'])
for i in range(len(config.unet)):
    config.unet[i]['preprocessing']= get_preprocessing(smp.encoders.get_preprocessing_fn(config.unet[i]['backbone'],"imagenet"))
    config.unet[i]['aug'] = get_valid_augmentation(config.unet[i]['im_size'])

In [ ]:
def norm(img):
    img = img.astype(float)
    img-=img.min()
    img/=img.max()
    img*=255
    return img.astype(np.uint8)

def get_image(n):
    path_img = f'{config.real_path}/{n[0]}/{n[1]}/{n[2]}/{n}.png'
    img = cv2.imread(path_img,0)
    if os.path.exists(f'{config.mask_path}/{n}.png'):
        path_mask = f'{config.mask_path}/{n}.png'
        mask = cv2.imread(path_mask)>(config.mask_TH*255)
        by_crop = False
    else:
        ratio = float(img.shape[0])/float(img.shape[1])
        if ratio > 3.5 or ratio < 1/3.5 or max(img.shape[0],img.shape[1])>550:
            mask = predict_one_image_with_crop(config_b3,img)
            mask = mask>(0.2)
            by_crop = True
        else:
            mask = predict_one_image(config_b3,img)
            mask = mask>config.mask_TH
            by_crop = False
        mask = mask[:,:,::-1]
    img = np.expand_dims(img,-1)
    img2 = np.concatenate([norm(mask),norm(img)],-1)
    
    return img2,by_crop

def get_Letters(n,imreal,img,verbose=False,TH=0.2,rot90=True):
    is_rot90 = False
#     imreal = f"{real_path}/{n[0]}/{n[1]}/{n[2]}/{n}.png"
#     imreal = cv2.imread(imreal,0)
#     path = f"{mask_path}/{n}.png"
#     img = mmcv.imread(path)
    img[:,:,0] = 255-imreal
    img = cv2.UMat(img).get()
    if img.shape[0]>img.shape[1] and rot90:
        img = cv2.UMat(np.rot90(img,1).astype(np.uint8)).get()
        imreal = np.rot90(imreal,1)
        is_rot90=True
    res = inference_detector(model, img,rot90=rot90)
    res[0][:,-1]*=5
    res[1][:,-1]*=2
    new = []
    label = []
    for cl,row in enumerate(res):
        for i in row:
            new.append(np.array(i))
            label.append(cl)
    new = np.array(new) 
    label = np.array(label) 
    index = nms(new[:,:4],new[:,4],0.2)
    final = np.concatenate([new[index],label[index].reshape(-1,1)],1)
    if verbose:
        for bbox in final:
            x0,y0,x1,y1,p,cl = bbox
            if p>TH:
                cv2.putText(img,str(config.classes[int(cl)]),(int(x0),int(y0)),cv2.FONT_HERSHEY_SIMPLEX,0.6 if img.shape[0]>300 else 0.45, (255, 255, 255), 1, cv2.LINE_AA)
    #             cv2.putText(img,str(round(p,1)),(int(x1)+6,int(y0)),font,0.6, (255, 255, 255), 1, cv2.LINE_AA)
                cv2.rectangle(img,(int(x0),int(y0)),(int(x1),int(y1)),config.colors[int(cl)])
        plt.figure(figsize=[20,20])
        plt.subplot(121)
        plt.imshow(imreal,cmap='gray')
        plt.subplot(122)
        plt.imshow(img)
        plt.show()
    return final[final[:,-2]>TH],is_rot90

def get_Neighbors_Atom(name,verbose=False,real_path="../input/bms-molecular-translation/test",mask_path='../input/mol-temp'):
    noise_img,by_crop = get_image(name)
    print(by_crop)
    Letters_cord,rot_90 = get_Letters(name,noise_img[:,:,-1].copy(),noise_img[:,:,:3].copy(),verbose=verbose,TH=config.bbox_TH)
    if rot_90:
        noise_img = np.rot90(noise_img,1)
    if verbose:
        plt.figure(figsize=[20,20])
        plt.subplot(141)
        plt.imshow(noise_img)
        plt.subplot(142)
        plt.imshow(noise_img[:,:,-1],cmap='gray')
        plt.subplot(143)
        plt.imshow(noise_img[:,:,1]-noise_img[:,:,0])
        plt.subplot(144)
        plt.imshow(noise_img[:,:,2])
        plt.show()
    im1 = noise_img[:,:,1]>0
    im_real = noise_img[:,:,-1]<127
    im0 = noise_img[:,:,0]>0.9
    im_real_bool = im_real.astype(float)-im0.astype(float)
    im_real_bool[im_real_bool<0]=0
    lbl_0 = label(im0) 
    props = regionprops(lbl_0)
    im = im1.astype(float)-im0.astype(float)
    im[im<0]=0
    lbl_1 = label(im) 
    props_1 = regionprops(lbl_1)
    lbl_0 = lbl_0.astype(np.uint8)
    r = np.array(Counter(lbl_0.reshape(-1)))
    if verbose:
        plt.figure(figsize=[20,20])
        plt.subplot(141)
        plt.imshow(lbl_0,cmap='hot')
        plt.subplot(142)
        plt.imshow(lbl_1,cmap='hot')
        plt.subplot(143)
        plt.imshow(im_real_bool,cmap='gray')
        plt.subplot(144)
        plt.imshow(im)
        plt.show()
    bbox = []
    area =[]
    dist ={}
    Atom = {}
    dist_min =[]
    font                   = cv2.FONT_HERSHEY_SIMPLEX
    fontScale              = 0.5
    fontColor              = (0,0,255)
    lineType               = 2
    # fontColor              = 255
    img = np.zeros([noise_img.shape[0],noise_img.shape[1],3])
    img[:,:,0] = noise_img[:,:,-1].copy()
    size_final = 0 
    x,y = props[0].centroid
    for i,prp in enumerate(props):
    #     if prp.area>5:
        c_flag = True
        for j,bbox in enumerate(Letters_cord):
            if bb_intersection_over_union([prp.bbox[1],prp.bbox[0],prp.bbox[3],prp.bbox[2]],bbox[0:4])>0:
                cl = bbox[-1]
                Atom[i]=int(config.classes[int(cl)])
                c_flag = False
                break
        if c_flag:
            Atom[i] = 6
        cv2.putText(img,str(i), 
            (prp.bbox[1],prp.bbox[0]), 
            font, 
            fontScale,
            fontColor,
            lineType)
        size_final+=1
        if i!=0:
            x1,y1 = prp.centroid
            d = distance.euclidean([x1,y1],[x,y])
            dist[i] = d
            dist_min.append(int(d))
    dist_min = np.sort(dist_min)[4]
    img_0 = np.zeros([lbl_0.shape[0],lbl_0.shape[1],3]).astype(np.uint8)
    print(lbl_0.max())
    for i,prop in enumerate(props):
        if prop.area>5:
            y,x = prop.centroid
            r = max(abs(prop.bbox[1]-prop.bbox[3]),abs(prop.bbox[0]-prop.bbox[2]))*1
            cv2.circle(img_0,(int(x),int(y)),int(r),(i,0,0),-1)
    d = np.zeros_like(img_0)
    img_0 = img_0[:,:,0]
    final = np.zeros([size_final,size_final])
    avg = []
    for row in range(lbl_1.max()):
        l = lbl_1.copy()
        l[l!=row+1]=0
        l[l!=0]=1
        res = img_0*l
        avg.append(res.sum())
    avg = np.median(avg)
    for row in range(lbl_1.max()+1):

        l = lbl_1.copy()
        l[l!=row+1]=0
        l[l!=0]=1
        res = img_0*l
        result = list(Counter(res.reshape(-1)))
        if len(result)==2:
            dist_new = dist[result[-1]]
            if dist_new>dist_min:
                continue
            final[result[0],result[1]]+=1
            final[result[1],result[0]]+=1
        elif len(result)==3:
            final[result[1],result[2]]+=1
            final[result[2],result[1]]+=1
    final[:,0][final[:,0]>2]=2
    final[0,:][final[0,:]>2]=2
    if verbose:
        plt.figure(figsize=(15, 15))
        plt.subplot(121)
        plt.imshow(final,cmap='hot')
        plt.subplot(122)
        plt.imshow(img)
        plt.show()
    return pd.DataFrame(final),pd.DataFrame.from_dict(Atom, orient="index",columns=['Atom_num'])

In [ ]:
for row in tqdm(range(0,20)):
    print(df.image_id[row])
    print(df['InChI'][row])
    n_table,ATOM = get_Neighbors_Atom(df.image_id[row],True,real_path=config.real_path,mask_path=config.mask_path)
    print("---------------------------------------------------------------")